Nesta atividade construiremos modelos preditivos de regressão em scikit learn para a predição dos votos de deputados federais considerando as últimas eleições. As atividades esperadas para essa etapa são descritas a seguir:

1. Baixe os dados aqui.

Vamos explorar dados sobre as votações que candidatos à Câmara Federal de Deputados receberam nos anos de 2006 e 2010. Esses dados foram extraídos do TSE (Links para um site externo)Links para um site externo, pré-processados e contemplam informações sobre aproximadamente 7.300 candidatos.

2. Considere o pipeline mostrado nesse [link](https://www.kaggle.com/apapiu/regularized-linear-models) para um site externo para construir seus modelos de regressão. Isso implica, dentre outras coisas:

    1. Analisar as distribuições das variáveis para ver se estão enviesadas e precisam de correção; tratamento de valores ausentes, variáveis categóricas e normalização, quando for o caso.
    2. Construir modelos de regressão com (ridge e lasso) e sem regularização.
    3. Considerar também modelos de regressão não paramétrica como K-NN.
    4. Considerar outros modelos ainda não vistos em sala de sua escolha (e.g. SVR, Regression Trees e Random Florests).
    5. Tunar os hiperâmetros para cada caso e retornar os rmses de validação cruzada para todos os modelos avaliados.
    6. Plotar os resíduos versus predições e analisar se esses plots representam bons indícios da adequabilidade dos modelos a esse problema.
    
3. Alguns dias antes da entrega final serão liberados os dados de teste referentes à 2014 para validação final dos seus melhores modelos.
    1. Dica: Uma coisa que você pode fazer é usar os dados de 2006 como treino e os de 2010 como validação. Uma vez encontrados os melhores modelos para 2010 junte 2006+2010, retreine, e aplique o modelo aos dados de 2014 que serão liberados.
    
4. Responder:
    1. Dentre os modelos avaliados, qual foi o que deu o melhor resultado nos dados de 2014 em termos de RMSE? Justifique bem sua resposta.

A entrega deve ser um notebook Jupyter com código python e texto explicativo quando necessário. Crie um repositório na sua conta do github e envie o link do html do notebook.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew

from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier

## Análise dos dados 

### Leitura dos dados

In [2]:
train_data = pd.read_csv("dados/eleicoes_2006_a_2010.csv")
test_data = pd.read_csv("dados/eleicoes_2014.csv")

# load the dataset as a pandas data frame
df_train_data = pd.DataFrame(train_data)
df_test_data = pd.DataFrame(test_data)

frames = [df_train_data, df_test_data]
df_data = pd.concat(frames)

In [3]:
df_data.head()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos/comites,...,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao,votos
0,2006,10001,JOSÉ LUIZ NOGUEIRA DE SOUSA,AP,PT,6,6,16600.00,2766.666667,NaN,...,14,14,16583.60,1184.542857,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR,8579
1,2006,10002,LOIVA DE OLIVEIRA,RO,PT,13,13,22826.00,1755.846154,6625.0,...,24,23,20325.99,883.738696,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL,2757
2,2006,10002,MARIA DALVA DE SOUZA FIGUEIREDO,AP,PT,17,16,148120.80,9257.550000,2250.0,...,123,108,146011.70,1351.960185,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO,17428
3,2006,10002,ROMALDO MILANI,MS,PRONA,6,6,3001.12,500.186667,NaN,...,8,8,3001.12,375.140000,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO INCOMPLETO,CASADO(A),MILITAR REFORMADO,1193
4,2006,10003,ANSELMO DE JESUS ABREU,RO,PT,48,48,NaN,NaN,NaN,...,133,120,116416.64,970.138667,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,CASADO(A),DEPUTADO,29401


### Pré-processamento dos dados

- Transformação das features numéricas enviesadas usando log(feature + 1), a fim de normalizar a distribuição dos valores;
- Tratamento das features numéricas com valores ausentes (NaN's);
- Transformação das variáveis categóricas.

#### Enviesamento de features numéricas

Através dos histogramas abaixo vemos que as variáveis analisadas precisam de normalização, porque a distribuição dos seus valores não está igual.

In [4]:
# function to plot a histogram of each variable
def plot_histogram(feature):
    # log (feature + 1) because of features with values 0
    feature = pd.DataFrame({"1. " + feature: df_data[feature], "2. log(" + feature + ")": np.log1p(df_data[feature])})
    feature.hist() # TODO check how to increase the number of bins

# function to transform skewed variables using log(feature + 1)
def transform_skewed_var(feature):
    df_data[feature] = np.log1p(df_data[feature])

In [5]:
columns = "quantidade_doacoes quantidade_doadores total_receita media_receita recursos_de_outros_candidatos/comites recursos_de_pessoas_fisicas recursos_de_pessoas_juridicas recursos_proprios quantidade_despesas quantidade_fornecedores total_despesa media_despesa".split()

for feature in columns:
    plot_histogram(feature)
    transform_skewed_var(feature)    

#### Features com valores numéricos ausentes (NaN) 

Abaixo veremos quais as variáveis com valores ausentes (NaN).

In [6]:
df_data.isnull().any()

ano                                      False
sequencial_candidato                     False
nome                                     False
uf                                       False
partido                                  False
quantidade_doacoes                       False
quantidade_doadores                      False
total_receita                             True
media_receita                             True
recursos_de_outros_candidatos/comites     True
recursos_de_pessoas_fisicas               True
recursos_de_pessoas_juridicas             True
recursos_proprios                         True
quantidade_despesas                      False
quantidade_fornecedores                  False
total_despesa                            False
media_despesa                            False
cargo                                    False
sexo                                     False
grau                                     False
estado_civil                             False
ocupacao     

In [7]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12266 entries, 0 to 4935
Data columns (total 23 columns):
ano                                      12266 non-null int64
sequencial_candidato                     12266 non-null int64
nome                                     12266 non-null object
uf                                       12266 non-null object
partido                                  12266 non-null object
quantidade_doacoes                       12266 non-null float64
quantidade_doadores                      12266 non-null float64
total_receita                            12105 non-null float64
media_receita                            12105 non-null float64
recursos_de_outros_candidatos/comites    8131 non-null float64
recursos_de_pessoas_fisicas              8374 non-null float64
recursos_de_pessoas_juridicas            5068 non-null float64
recursos_proprios                        7802 non-null float64
quantidade_despesas                      12266 non-null float64
quantid

Das 6 variáveis com valores ausentes,  **recursos_de_outros_candidatos/comites**, **recursos_de_pessoas_fisicas**, **recursos_de_pessoas_juridicas** e **recursos_proprios** são as que mais possuem valores ausentes.

O gráfico mais abaixo mostra a correlação entre todas as variáveis numéricas. Nele, quanto mais amarelo maior a correlação entre a variável do eixo horizontal e a variável do eixo vertical.

In [8]:
#analysis of correlation between variables

corr = df_data.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
cmap='viridis'

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=0.4, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Vemos acima que a variável que mais tem correlação com as outras é **total_receita**.

Apesar dela também ter valores ausentes, optou-se por remover dos dados apenas as outras 4 variáveis que tem mais valores ausentes, mas tem alta correlação com **total_receita**. Nesta última e em **media_receita**, os valores ausentes serão substituídos pela mediana de todos os valores de cada coluna.

In [9]:
print(df_data.shape)

#removing collumns
df_data = df_data.drop(["nome", "sequencial_candidato"], 1)

#filling NA's of total_receita and media_receita with the median of the column:
df_data = df_data.fillna(df_data.median())

print(df_data.shape)
df_data.isnull().any()

(12266, 23)
(12266, 21)


ano                                      False
uf                                       False
partido                                  False
quantidade_doacoes                       False
quantidade_doadores                      False
total_receita                            False
media_receita                            False
recursos_de_outros_candidatos/comites    False
recursos_de_pessoas_fisicas              False
recursos_de_pessoas_juridicas            False
recursos_proprios                        False
quantidade_despesas                      False
quantidade_fornecedores                  False
total_despesa                            False
media_despesa                            False
cargo                                    False
sexo                                     False
grau                                     False
estado_civil                             False
ocupacao                                 False
votos                                    False
dtype: bool

#### Tranformação das variáveis categóricas

As variáveis categóricas presentes nos dados, como **ocupação** e **estado_civil**, serão transformadas em variáveis numéricas, a fim de permitir a sua utilização nos modelos de regressão logo a seguir.

In [10]:
df_data = pd.get_dummies(df_data)

df_data.head(10)

,ano,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos/comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,quantidade_despesas,...,ocupacao_TÉCNICO DE QUÍMICA,ocupacao_TÉCNICO EM AGRONOMIA E AGRIMENSURA,ocupacao_TÉCNICO EM EDIFICAÇÕES,ocupacao_TÉCNICO EM INFORMÁTICA,ocupacao_VENDEDOR DE COMÉRCIO VAREJISTA E ATACADISTA,"ocupacao_VENDEDOR PRACISTA, REPRESENTANTE, CAIXEIRO-VIAJANTE E ASSEMELHADOS",ocupacao_VEREADOR,ocupacao_VETERINÁRIO,ocupacao_VIGILANTE,ocupacao_ZOOTECNISTA
0,2006,1.945910,1.945910,9.717218,7.925760,8.225474,9.105091,8.748464,7.170888,2.708050,...,0,0,0,0,0,0,1,0,0,0
1,2006,2.639057,2.639057,10.035699,7.471276,8.798757,9.615872,6.908755,5.308268,3.218876,...,0,0,0,0,0,0,0,0,0,0
2,2006,2.890372,2.833213,11.905790,9.133303,7.719130,10.438547,11.038461,10.993748,4.820282,...,0,0,0,0,0,0,0,0,0,0
3,2006,1.945910,1.945910,8.007074,6.216979,8.225474,7.048386,7.004991,6.621406,2.197225,...,0,0,0,0,0,0,0,0,0,0
4,2006,3.891820,3.891820,9.375951,7.307409,8.225474,10.837206,10.181817,11.141035,4.897840,...,0,0,0,0,0,0,0,0,0,0
5,2006,1.945910,1.945910,7.479983,5.691041,8.225474,8.794976,10.181817,7.479983,2.302585,...,0,0,0,0,0,0,0,0,0,0
6,2006,2.708050,2.079442,6.548219,4.610868,8.225474,7.116394,10.181817,8.517393,2.890372,...,0,0,0,0,0,0,0,0,0,0
7,2006,1.098612,1.098612,11.156265,10.463132,8.225474,8.794976,10.181817,11.156265,5.808142,...,0,0,0,0,0,0,0,0,0,0
8,2006,1.098612,1.098612,8.710455,8.017472,8.710455,8.794976,10.181817,8.517393,2.397895,...,0,0,0,0,0,0,0,0,0,0
9,2006,4.234107,4.189655,11.569440,7.395657,9.167015,10.987734,10.165890,9.711176,4.234107,...,0,0,0,0,0,0,0,0,0,0


### Divisão dos dados em treinamento, validação e teste

Abaixo, o conjunto de dados utilizado será particionado em treino, para servir para a criação dos modelos de regressão, em validação, que será usado para avaliar a qualidade do modelo e em teste, que servirá para avaliar as predições do modelo para dados ainda não vistos por ele.

In [11]:
data_train = df_data.loc[df_data["ano"] == 2006]
y_train = data_train.votos
X_train = data_train.drop("votos", 1)
print(X_train.shape)
print(y_train.shape)

data_validation = df_data.loc[df_data["ano"] == 2010]
y_validation = data_validation.votos
X_validation = data_validation.drop("votos", 1)
print(X_validation.shape)
print(y_validation.shape)

data_test = df_data.loc[df_data["ano"] == 2014]
y_test = data_test.votos
X_test = data_test.drop("votos", 1)
print(X_test.shape)
print(y_test.shape)

(3388, 283)
(3388,)
(3942, 283)
(3942,)
(4936, 283)
(4936,)


## Implementação dos Modelos de regressão

### 1. Modelos de regressão sem regularização

In [12]:
# function to calculate the RMSE of the model with cross validation
def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring = "neg_mean_squared_error", cv = 10))
    return(rmse)

In [13]:
def print_coefficients(model):
    w = list(model.coef_)
    # Parameters must be in reverse order because the poly function 
    w.reverse()
    print (np.poly1d(w) + model.intercept_) 

In [14]:
def plot_rmse_param(series, param_name):
    series.plot(title = "Erro de validação vs " + param_name)
    plt.xlabel(param_name)
    plt.ylabel("rmse")

In [15]:
def best_rmse_param(series):
    best_rmse = series.min()
    best_param = series.idxmin() 
    
    return(best_rmse, best_param)

In [16]:
def train_model(model, print_coef):
    model.fit(X_train, y_train)
    if (print_coef):
        print_coefficients(model)
        
    print("Score: ", model.score(X_validation, y_validation)) #TODO change to X_test, y_test

#### 1.1 Implementando o modelo de regressão Ridge

In [17]:
model_ridge = Ridge()
train_model(model_ridge, True)

            282             281        280        279        278
-1.066e+04 x   + 1.189e+04 x   + 2355 x   - 3747 x   - 3981 x  
         277        276             275        274        272
 - 4067 x   - 2613 x   + 9.081e+04 x   - 3554 x   - 5719 x  
              271         269         268        267        266
 - 1.477e+04 x   - 974.8 x   - 549.4 x   + 2765 x   - 1147 x  
         265             263        262             261             256
 - 3675 x   + 2.237e+04 x   - 2646 x   - 1.787e+04 x   - 1.103e+04 x  
              255            253        252        251        250
 + 2.314e+04 x   - 1.05e+04 x   - 4921 x   + 9007 x   + 5954 x  
            249        248        247        246        245        244
 - 1.5e+04 x   + 2504 x   - 4840 x   - 5049 x   - 1668 x   - 1687 x  
         243        241             239        237         236
 - 8002 x   + 1292 x   + 1.136e+04 x   - 2489 x   - 526.9 x  
         235        232        231        230        229        228
 - 5016 x   -

O hiperparâmetro do Rigde é o alpha - um parâmetro de regularização que mede quão flexível é o modelo. Quanto maior a regularização, menos propenso será o nosso modelo de overfitting. No entanto, também perderá a flexibilidade e poderá não capturar todo o padrão dos dados.

Testaremos alguns valores para alpha e usaremos o melhor para treinar o modelo.

In [18]:
alphas = [0.001, 0.1, 0.5, 1, 5, 10, 15, 20, 25]
cv_ridge_rmse = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]

series_ridge = pd.Series(cv_ridge_rmse, index = alphas)
plot_rmse_param(series_ridge, "alpha")

Observe a curva em forma de U acima. Quando o alpha é muito grande, a regularização é muito forte e o modelo não consegue capturar todas as complexidades dos dados - o erro volta a aumentar. Se, no entanto, deixarmos o modelo ser muito flexível (alpha pequeno), o modelo tende ao overfitting. Um valor de alpha = 30 parece o melhor, com base no gráfico acima.

In [19]:
best_rmse_ridge, best_param_ridge = best_rmse_param(series_ridge)
print(best_rmse_ridge, best_param_ridge)

31436.9030139 10.0


In [20]:
model_ridge = Ridge(alpha = best_param_ridge)
train_model(model_ridge, True)

       282        281        280        279        278        277
-2033 x   + 5423 x   + 1460 x   - 3448 x   - 1485 x   - 1925 x  
          276             275         274        272        271
 + 413.1 x   + 1.659e+04 x   - 721.4 x   - 1167 x   - 2196 x  
          269         268        267         266        265        263
 - 91.41 x   + 277.5 x   + 1250 x   - 257.8 x   - 2204 x   + 9750 x  
          262        261        256        255        253         252
 - 695.6 x   - 3186 x   - 2198 x   + 9014 x   - 2254 x   - 876.8 x  
         251        250        249         248        247        246
 + 3515 x   + 1340 x   - 2493 x   + 252.7 x   - 2267 x   - 2982 x  
         245        244        243         241        239         237
 - 1336 x   - 1253 x   - 6143 x   + 369.7 x   + 2335 x   - 562.8 x  
          236        235        232        231         230        229
 - 323.6 x   - 3217 x   - 2027 x   - 3650 x   - 184.6 x   + 1553 x  
         228         227        226        225 

#### 1.2 Implementando o modelo de regressão Lasso

In [21]:
model_lasso = Lasso(max_iter = 10000)
train_model(model_lasso, True)

            282             281        280        279        278
-1.584e+04 x   + 1.495e+04 x   + 3830 x   - 1993 x   - 2554 x  
         277        276             275        274        272
 - 2415 x   - 1497 x   + 1.809e+05 x   - 1807 x   - 5236 x  
              271         268        267        265             263
 - 2.556e+04 x   + 159.3 x   + 4237 x   - 1834 x   + 2.748e+04 x  
          262             261            256             255
 - 283.1 x   - 3.108e+04 x   - 1.37e+04 x   + 2.898e+04 x  
              253        252           251        250            249
 - 1.522e+04 x   - 4867 x   + 1.2e+04 x   + 9563 x   - 2.58e+04 x  
         248        247        246         244        243         241
 + 3621 x   - 3465 x   - 3599 x   - 30.66 x   - 6506 x   + 706.4 x  
              239         237        236        235        232
 + 2.044e+04 x   - 613.1 x   + 1078 x   - 3410 x   - 8488 x  
         231         230             229        228        226
 - 6362 x   - 727.8 x   + 1.

Vamos agora usar o Lasso CV para descobrir o melhor alpha para nós.

In [22]:
alphas = [30, 35, 40, 45, 50]
cv_lasso_rmse = [rmse_cv(Lasso(alpha = alpha, max_iter = 10000)).mean() 
            for alpha in alphas]

series_lasso = pd.Series(cv_lasso_rmse, index = alphas)
plot_rmse_param(series_lasso, "alpha")

In [23]:
best_rmse_lasso, best_param_lasso = best_rmse_param(series_lasso)
print(best_rmse_lasso, best_param_lasso)

31398.1552074 35


In [24]:
model_lasso = Lasso(alpha = best_param_lasso, max_iter = 10000)
train_model(model_lasso, True)

       279            275        263        255         246        243
-1576 x   + 6.61e+04 x   + 7563 x   + 4839 x   - 51.38 x   - 3334 x  
         222        219         211        202        201        154
 - 1254 x   + 1964 x   + 265.6 x   + 1328 x   + 2645 x   + 7825 x  
              138         118         92        91         89        88
 + 2.906e+04 x   - 791.6 x   + 72.61 x  + 3183 x  + 579.8 x  + 1052 x 
         87         84         81        80        77        74
 - 1246 x  - 464.9 x  + 520.7 x  + 1485 x  - 1789 x  - 5479 x 
         72        71        69        67        65        64        62
 + 3247 x  - 4553 x  - 4097 x  + 1839 x  - 1026 x  + 2621 x  - 1546 x 
          61        57        54        52        51        50
 - 284.9 x  + 2571 x  - 4856 x  + 6128 x  + 1352 x  + 4565 x 
         49        47         45             39        38        37
 - 2419 x  + 4583 x  - 183.3 x  - 1.404e+04 x  + 1587 x  + 1392 x 
          36        34        33             32  

In [25]:
coef = pd.Series(model_lasso.coef_, index = X_train.columns)
print("Lasso escolheu " + str(sum(coef != 0)) + " variáveis and eliminou as outras " +  str(sum(coef == 0)) + " variáveis.")

Lasso escolheu 74 variáveis and eliminou as outras 209 variáveis.


### 2. Modelos de regressão não parámetrica

#### 2.1 Implementando o modelo de regressão K-NN

In [26]:
model_knn = KNeighborsClassifier()
train_model(model_knn, False)

Score:  0.00101471334348


A quantidade de vizinhos (k) é o parâmetro mais importante do algoritmo K-NN e define a quantidade de valores vizinhos de cada valor de entrada que o algoritmo deve considerar para estimar a saída. Por isso, usaremos cross-validation para encontrar o melhor valor deste parâmetro.

In [36]:
n_neighbors = [3, 5, 10, 25, 50]
cv_knn_rmse = [rmse_cv(KNeighborsClassifier(n_neighbors = n)).mean() 
            for n in n_neighbors]

series_knn = pd.Series(cv_knn_rmse, index = n_neighbors)
plot_rmse_param(series_knn, "n_neighbors")

In [ ]:
best_rmse_knn, best_param_knn = best_rmse_param(series_knn)
print(best_rmse_knn, best_param_knn)

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors = best_param_knn)
train_model(model_knn, False)

### 3. Outros modelos de regressão

#### 3.1 Implementando o modelo SVR (Suport Vector Regression)

In [37]:
model_svr = SVR()
train_model(model_svr, False)

Score:  -0.134050656861


O C é um parâmetro de penalidade importante no SVR, porque se for muito grande poderemos ter overffiting ou underffiting, caso contrário. Por isso, tentaremos encontrar o seu melhor valor.

In [38]:
cs = [1, 5, 10, 25, 50]
cv_svr_rmse = [rmse_cv(SVR(C = c)).mean() 
            for c in cs]

series_svr = pd.Series(cv_svr_rmse, index = cs)
plot_rmse_param(series_svr, "C")

In [39]:
best_rmse_svr, best_param_svr = best_rmse_param(series_svr)
print(best_rmse_svr, best_param_svr)

43181.3598384 50


In [40]:
model_svr = SVR(C = best_param_svr)
train_model(model_svr, False)

Score:  0.0734340899824


#### 3.2 Implementando o modelo de regressão Random Forest

In [41]:
model_rf = RandomForestClassifier()
train_model(model_rf, False)

Score:  0.00025367833587


Um dos parâmetros mais importantes do Random Forest é a quantidade de estimadores, que representam a quantidade de árvores usadas no modelo. Em geral, quanto maior melhor o desempenho do modelo, porém mais memória é necessário.

Abaixo, encontramos o melhor valor dado um intervalo de 1 à 20.

In [43]:
n_estimators = [1, 5, 10, 15, 20]
cv_rf_rmse = [rmse_cv(RandomForestClassifier(n_estimators = n)).mean() 
            for n in n_estimators]

series = pd.Series(cv_rf_rmse, index = n_estimators)
plot_rmse_param(series, "n_estimators")

In [ ]:
best_rmse_rf, best_param_rf = best_rmse_param(series)
print(best_rmse_rf, best_param_rf)

In [ ]:
model_rf = RandomForestClassifier(n_estimators = best_param_rf)
train_model(model_rf, False)

## Análise da predição dos modelos

In [44]:
# function to plot the residual vs predictions values of the model
def plot_res_vs_pred(model):
    y_pred = model.predict(X_validation)
    res = y_validation - y_pred
    #y_pred = model.predict(X_test)
    #res = Y_test - y_pred
    plt.plot(y_pred, res, 'k.', color='blue')
    plt.axhline(y=0., color='r', linestyle='-')
    plt.xlabel("predictions")
    plt.ylabel("residuals")

### Ridge

In [45]:
plot_res_vs_pred(model_ridge)

### Lasso

In [46]:
plot_res_vs_pred(model_lasso)

### K-NN

In [ ]:
plot_res_vs_pred(model_knn)

### SVR

In [47]:
plot_res_vs_pred(model_svr)

### Random Forest

In [ ]:
plot_res_vs_pred(model_rf)

## Conclusões